# Assignment 5 Scrapping Data
### Kevin Williams 

# Imports 

In [1]:
import pandas as pd
from requests_html import HTMLSession
import os
import re
from time import sleep
from tqdm.notebook import tqdm # progress bar
from NEAR_regex import NEAR_regex 
from bs4 import BeautifulSoup


0
1
0
0
0
1
0
0
1


In [2]:
#### SECTION 1: get the data that can build the urls)

if not os.path.exists('input/2007_inv_and_tech.dta'):

    # grab ccm from lehigh DA Github
    ccm = pd.read_stata('https://github.com/LeDataSciFi/lectures-spr2020/blob/master/assignment_data/2007_inv_and_tech.dta?raw=true')
    # save to my computer    
    ccm.to_stata('input/2007_inv_and_tech.dta')
    
else:
    
    # load from my computer
    ccm = pd.read_stata('input/2007_inv_and_tech.dta')
    

In [3]:
#### SECTION 2: crawl the urls

session = HTMLSession()
for row in tqdm(ccm.index, desc='Crawling in progress'): 
    row = ccm.iloc[row]
    
    if row['FName'] != '': 
        url      = 'https://www.sec.gov/Archives/' + row['FName']
        # Allows 10-k form derivaitves to be recognized as independent folders
        form     = row['Form'].replace('/','').replace('\\','')  
        # F-strings: [f'] allows you to print embed python expressions inside string literals for formatting
        CoName   = row['CoName'].replace('/','')
        folder   =  f'edgar_filings/{CoName}/{form}/fyear_{int(row["fyear"])}/'
        filename = row['FName'].split('/')[-1] 
        filepath = folder + filename
        
        if not os.path.exists(folder):
            os.makedirs(folder)
        if not os.path.exists(filepath):
            try:
                r = session.get(url)
            except:
                print('failure to get url:',url)  #log error 
            else:   
                with open(filepath,'w',encoding='utf8') as f:
                    f.write(r.text)
                sleep(2)    
